# notebook for create init and true test model

In [1]:
# read crust1.0 model

import os
import math
import numpy as np

N_all_lon = 360
d_all_lon = 1
c1_all_lon = []
for i in range(N_all_lon):
    lon = i - 179.5
    c1_all_lon.append(lon)

N_all_lat = 180
d_all_lat = -1
c1_all_lat = []
for i in range(N_all_lat):
    lat = 89.5 - i
    c1_all_lat.append(lat)


def read_crust1p0(path):
    N_all_lat = 180
    N_all_lon = 360
    layer = np.full((9,N_all_lat,N_all_lon),-1.0)
    for i in range(9):
        filename = '%s/xyz-bd%d'%(path,i+1)
        doc = open(filename,'r')
        doc_data = doc.readlines()
        doc.close()

        for ilat in range(N_all_lat):
            for ilon in  range(N_all_lon):
                tmp = doc_data[ilon + ilat * N_all_lon].split()
                layer[i,ilat,ilon] = -float(tmp[2])

    vp = np.full((9,N_all_lat,N_all_lon),-1.0)
    for i in range(9):
        filename = '%s/xyz-vp%d'%(path,i+1)
        doc = open(filename,'r')
        doc_data = doc.readlines()
        doc.close()

        for ilat in range(N_all_lat):
            for ilon in  range(N_all_lon):
                tmp = doc_data[ilon + ilat * N_all_lon].split()
                vp[i,ilat,ilon] = float(tmp[2])


    vs = np.full((9,N_all_lat,N_all_lon),-1.0)
    for i in range(9):
        filename = '%s/xyz-vs%d'%(path,i+1)
        doc = open(filename,'r')
        doc_data = doc.readlines()
        doc.close()

        for ilat in range(N_all_lat):
            for ilon in  range(N_all_lon):
                tmp = doc_data[ilon + ilat * N_all_lon].split()
                vs[i,ilat,ilon] = float(tmp[2])


    return (layer,vp,vs)

(layer,vp,vs) = read_crust1p0('reference_models/crust1p0')


In [2]:
# read 1D model, e.g., ak135 model

def read_1d_model(filename):
    doc = open(filename,'r')
    doc_data = doc.readlines()
    doc.close()

    layer = []
    vp = []
    vs = []
    for i,info in enumerate(doc_data):
        if (i<0.5):
            continue
        
        tmp = info.split()
        layer.append(float(tmp[0]))
        vp.append(float(tmp[1]))
        vs.append(float(tmp[2]))
    return (layer,vp,vs)

(layer_1d,vp_1d,vs_1d) = read_1d_model('reference_models/ak135.model')


In [3]:
# function : find adjacent points
def find_adjacent_point(point,array):

    id1 = -2; val1 =  10000000 
    id2 = -1; val2 = -10000000
    narray = len(array)
    for i in range(narray):
        dis = point - array[i]
        if (dis >= 0 and dis < val1):
            id1 = i; val1 = dis

        if (dis < 0 and dis > val2):
            id2 = i; val2 = dis

    r1 = (point - array[id2])/(array[id1] - array[id2])
    if(abs(array[id1]-array[id2])<0.01):
        print(point,id1,id2,array)
    return (r1,id1,id2)

In [4]:
# crust 1.0 model 1 degree by 1 degree
out_path = 'models'
try:
    os.mkdir(out_path)
except:
    print('%s exist'%(out_path))
all_vp = []
all_dep = []
for lat_id in range(N_all_lat):
    tmp_vp_lon = []
    tmp_dep_lon = []
    for lon_id in range(N_all_lon):
        tmp_vp = []
        tmp_dep = []
        for i in range(3,9):
            if (vp[i,lat_id,lon_id] > 0.1):
                tmp_vp.append(vp[i,lat_id,lon_id])
                tmp_dep.append(layer[i,lat_id,lon_id])
        
        tmp_vp_lon.append(tmp_vp)
        tmp_dep_lon.append(tmp_dep)

    all_vp.append(tmp_vp_lon)
    all_dep.append(tmp_dep_lon)



doc = open('%s/crust1.0_layer.dat'%out_path,'w')

for lat_id in range(N_all_lat):
    for lon_id in range(N_all_lon):
        lat = c1_all_lat[lat_id]
        lon = c1_all_lon[lon_id]
        doc.write('%8.3f %8.3f '%(lat,lon))
        for ilayer in range(len(all_dep[lat_id][lon_id])):
            doc.write('%8.3f(%8.3f) '%(all_dep[lat_id][lon_id][ilayer],all_vp[lat_id][lon_id][ilayer]))
        doc.write('\n')
doc.close()




models exist


In [5]:
# build initial model

import numpy as np
import math

# grid
R_earth = 6371.0 

rr1=5821.0
rr2=6421.0
tt1=(10.5)/180*math.pi
tt2=(22.5)/180*math.pi
pp1=(95.5)/180*math.pi
pp2=(107.5)/180*math.pi

n_rtp = [81,81,81]
dr = (rr2-rr1)/(n_rtp[0]-1)
dt = (tt2-tt1)/(n_rtp[1]-1)
dp = (pp2-pp1)/(n_rtp[2]-1)
rr = np.array([rr1 + x*dr for x in range(n_rtp[0])])
tt = np.array([tt1 + x*dt for x in range(n_rtp[1])])
pp = np.array([pp1 + x*dp for x in range(n_rtp[2])])



eta_init = np.zeros(n_rtp)
xi_init  = np.zeros(n_rtp)
zeta_init = np.zeros(n_rtp)
fun_init = np.zeros(n_rtp)
vel_init = np.zeros(n_rtp)


c=0

all_ratio_id_t = []
for it in range(n_rtp[1]):
    t = tt[it] * 180/math.pi
    (ratio_t,idt1,idt2)=find_adjacent_point(t,c1_all_lat)
    all_ratio_id_t.append([ratio_t,idt1,idt2])

all_ratio_id_p = []    
for ip in range(n_rtp[2]):
    p = pp[ip] * 180/math.pi
    (ratio_p,idp1,idp2)=find_adjacent_point(p,c1_all_lon)
    all_ratio_id_p.append([ratio_p,idp1,idp2])

all_ratio_id_r = []
for ir in range(n_rtp[0]):
    dep = R_earth - rr[ir]
    (r3,idr1,idr2) = find_adjacent_point(dep,layer_1d)
    all_ratio_id_r.append([r3,idr1,idr2])

gamma = 0.0



for it in range(n_rtp[1]):
    (ratio_t,idt1,idt2)=all_ratio_id_t[it]
    for ip in range(n_rtp[2]):
        (ratio_p,idp1,idp2)=all_ratio_id_p[ip]


        for ir in range(n_rtp[0]):
            dep = R_earth - rr[ir]

            grid_vp = 0.0; 
            grid_vs = 0.0; 

            # consider four 1d models at points adjacent to the forward grid (c1_all_lat[idt1 or idt2],c1_all_lon[idp1 or idp2])
            for i_grid in range(0,4):
                if(i_grid == 0):
                    lat_id = idt1; lon_id = idp1; r1 = ratio_t; r2 = ratio_p; 
                elif(i_grid == 1):
                    lat_id = idt2; lon_id = idp1; r1 = 1-ratio_t; r2 = ratio_p; 
                elif(i_grid == 2):
                    lat_id = idt1; lon_id = idp2; r1 = ratio_t; r2 = 1-ratio_p; 
                elif(i_grid == 3):
                    lat_id = idt2; lon_id = idp2; r1 = 1-ratio_t; r2 = 1-ratio_p; 

                # print(lat_id,lon_id,all_dep[lat_id][lon_id][-1])

                        
                if (dep < all_dep[lat_id][lon_id][0]):
                    tmp_vp = all_vp[lat_id][lon_id][0]
                elif (dep < all_dep[lat_id][lon_id][-1]):             
                    (r3,idr1,idr2) = find_adjacent_point(dep,all_dep[lat_id][lon_id])

                    # linear interpolation
                    tmp_vp = r3*all_vp[lat_id][lon_id][idr1]+(1-r3)*all_vp[lat_id][lon_id][idr2]    

                    # layered model
                    # tmp_vp = min(all_vp[lat_id][lon_id][idr1],all_vp[lat_id][lon_id][idr2])       
                else:
                 # combine crust 1.0 model and 1D model 
                    tmp_vp1 = all_vp[lat_id][lon_id][-1]  # crust1.0 模型

                    (r3,idr1,idr2) = all_ratio_id_r[ir]
                    tmp_vp2 = r3*vp_1d[idr1] + (1-r3) * vp_1d[idr2]
                    tmp_vp2 = max(tmp_vp2,8.04)

                    # linear interpolation (45 km 为分界线。之间插值，之后依靠一维模型)
                    if (dep < 45):
                        ratio = (dep - all_dep[lat_id][lon_id][-1])/(45 - all_dep[lat_id][lon_id][-1])
                        tmp_vp = tmp_vp1 * (1- ratio) + tmp_vp2 * ratio
                    else:
                        tmp_vp = tmp_vp2

                    # # linear interpolation
                    # if(tmp_vp1 > tmp_vp2):
                    #     tmp_vp = tmp_vp1
                    # else:
                    #     if(dep > all_dep[lat_id][lon_id][-1] + 100):
                    #         tmp_vp = tmp_vp2
                    #     else:
                    #         ratio = (dep - all_dep[lat_id][lon_id][-1])/100
                    #         tmp_vp = tmp_vp1 * (1- ratio) + tmp_vp2 * ratio

                    

                    # layered model
                    # tmp_vp = max(tmp_vp1,tmp_vp2)     

                grid_vp = grid_vp + tmp_vp * r1 * r2


            eta_init[ir,it,ip] = 0.0
            xi_init[ir,it,ip]  = 0.0
            zeta_init[ir,it,ip] = gamma*math.sqrt(eta_init[ir,it,ip]**2 + xi_init[ir,it,ip]**2)
            fun_init[ir,it,ip] = 1.0/grid_vp
            vel_init[ir,it,ip] = grid_vp
            

            
r_earth = R_earth #6378.1370
print("depminmax {} {}".format(r_earth-rr1,r_earth-rr2))
print(c)


depminmax 550.0 -50.0
0


In [6]:
# build checkerboard (target) model
# initial model

# 正式检测板测试1

# true model
eta_true = np.zeros(n_rtp)
xi_true  = np.zeros(n_rtp)
zeta_true = np.zeros(n_rtp)
fun_true = np.zeros(n_rtp)
vel_true = np.zeros(n_rtp)


for it in range(n_rtp[1]):
    lat = tt[it]/math.pi*180.0

    for ip in range(n_rtp[2]):
        lon = pp[ip]/math.pi*180.0

        for ir in range(n_rtp[0]):
            dep = R_earth - rr[ir]

            if(dep >=0 and dep < 100):
                sigma = 0.04*math.sin(math.pi*(lon-97)/1.5) * math.sin(math.pi*(lat-12)/1.5) * math.sin(math.pi*(dep)/100.0)
            elif(dep >=100 and dep <= 200):
                sigma = 0.04*math.sin(math.pi*(lon-97)/1.5) * math.sin(math.pi*(lat-12)/1.5) * math.sin(math.pi*(dep-100)/100.0+math.pi)
            elif(dep >200 and dep <= 350):
                sigma = 0.03*math.sin(math.pi*(lon-97)/1.5) * math.sin(math.pi*(lat-12)/1.5) * math.sin(math.pi*(dep-200)/150.0)
            elif(dep >350 and dep <= 500):
                sigma = 0.02*math.sin(math.pi*(lon-97)/1.5) * math.sin(math.pi*(lat-12)/1.5) * math.sin(math.pi*(dep-350)/150.0+math.pi)
            else:
                sigma = 0

            eta_true[ir,it,ip] = eta_init[ir,it,ip]
            xi_true[ir,it,ip]  = xi_init[ir,it,ip]
            zeta_true[ir,it,ip] = gamma*math.sqrt(eta_true[ir,it,ip]**2 + xi_true[ir,it,ip]**2)
            fun_true[ir,it,ip] = fun_init[ir,it,ip]/(1+sigma)
            vel_true[ir,it,ip] = 1.0/fun_true[ir,it,ip]    


In [7]:
# write out
import h5py

fout_init = h5py.File('%s/model_init_N%d_%d_%d.h5'%(out_path,n_rtp[0],n_rtp[1],n_rtp[2]), 'w')
fout_true = h5py.File('%s/model_ckb_N%d_%d_%d.h5'%(out_path,n_rtp[0],n_rtp[1],n_rtp[2]), 'w')

# write out the arrays eta_init, xi_init, zeta_init, vel_init,
fout_init.create_dataset('eta',  data=eta_init)
fout_init.create_dataset('xi',    data=xi_init)
fout_init.create_dataset('zeta',data=zeta_init)
fout_init.create_dataset('vel',  data=vel_init)

# writeout the arrays eta_true, xi_true, zeta_true, vel_true
fout_true.create_dataset('eta',  data=eta_true)
fout_true.create_dataset('xi',    data=xi_true)
fout_true.create_dataset('zeta',data=zeta_true)
fout_true.create_dataset('vel',  data=vel_true)

fout_init.close()
fout_true.close()
